In [1]:
!pip install elasticsearch

In [6]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")
es.info().body

{'name': '4b3c48fd7a26',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'udS8LAh9SrGEj8R5-bY5yg',
 'version': {'number': '8.7.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'f229ed3f893a515d590d0f39b05f68913e2d9b53',
  'build_date': '2023-04-27T04:33:42.127815583Z',
  'build_snapshot': False,
  'lucene_version': '9.5.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

In [7]:
es.cat.count(index="cpd",format="json")

ListApiResponse([{'epoch': '1684731780', 'timestamp': '05:03:00', 'count': '401'}])

In [13]:
mapping = es.indices.get_mapping(index="cpd")
mapping

ObjectApiResponse({'cpd': {'mappings': {'properties': {'article_title': {'type': 'text', 'analyzer': 'standard'}, 'main_section': {'type': 'text', 'analyzer': 'standard'}, 'page_id': {'type': 'integer'}, 'section_number': {'type': 'integer'}, 'section_title': {'type': 'text', 'analyzer': 'english'}, 'source_url': {'type': 'text', 'analyzer': 'standard'}, 'tags': {'type': 'text'}, 'text': {'type': 'text', 'analyzer': 'standard'}}}}})

In [55]:
res = es.get(index="cpd", id=1)
res

ObjectApiResponse({'_index': 'cpd', '_id': '1', '_version': 1, '_seq_no': 1, '_primary_term': 1, 'found': True, '_source': {'section_title': 'Definition', 'text': 'Pandemic,Definition\nA pandemic is an epidemic occurring on a scale that crosses international boundaries, usually affecting people on a worldwide scale. A disease or condition is not a pandemic merely because it is widespread or kills many people; it must also be infectious. For instance, cancer is responsible for many deaths but is not considered a pandemic because the disease is neither infectious nor contagious.', 'main_section': 'Definition', 'article_title': 'Pandemic', 'source_url': 'https://en.wikipedia.org/wiki/Pandemic', 'page_id': 24255, 'tags': 'Pandemic,Definition', 'section_number': 1}})

In [79]:
#Query using match
query_body={'query':{'match':{'text':'World Health Organization'}},'fields':['article_title','section_number'],'_source':False}
res=es.search(index="cpd", body=query_body)

/var/folders/84/skgmh_gx5bx_zpl3v_kjpwym0000gn/T/ipykernel_43750/2776792895.py:3: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res=es.search(index="cpd", body=query_body)


In [80]:
res['hits']['total']['value']

166

In [50]:
res

ObjectApiResponse({'took': 5, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 13, 'relation': 'eq'}, 'max_score': 3.6607487, 'hits': [{'_index': 'cpd', '_id': '46', '_score': 3.6607487, 'fields': {'article_title': ['Antonine Plague'], 'section_number': [0]}}, {'_index': 'cpd', '_id': '47', '_score': 3.6607487, 'fields': {'article_title': ['Antonine Plague'], 'section_number': [1]}}, {'_index': 'cpd', '_id': '48', '_score': 3.6607487, 'fields': {'article_title': ['Antonine Plague'], 'section_number': [3]}}, {'_index': 'cpd', '_id': '49', '_score': 3.6607487, 'fields': {'article_title': ['Antonine Plague'], 'section_number': [4]}}, {'_index': 'cpd', '_id': '50', '_score': 3.6607487, 'fields': {'article_title': ['Antonine Plague'], 'section_number': [5]}}, {'_index': 'cpd', '_id': '51', '_score': 3.6607487, 'fields': {'article_title': ['Antonine Plague'], 'section_number': [6]}}, {'_index': 'cpd', '_id': '52', '_score': 

In [81]:
#Query using term
query_body1={'query':{'term':{'article_title':'Antonine Plague'}},'fields':['article_title','section_number'],'_source':False}
res1=es.search(index="cpd", body=query_body1)

/var/folders/84/skgmh_gx5bx_zpl3v_kjpwym0000gn/T/ipykernel_43750/1934783922.py:3: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res1=es.search(index="cpd", body=query_body1)


In [82]:
res1['hits']['total']['value']

0

In [74]:
query_body2={'query':
                 {'bool':
                      {'must':
                           {'match':{'text':'World Health Organization'}},
                      'must_not':
                           {'match':{'section_title':'External links'}}}
                 }
            }
res2=es.search(index="cpd",body=query_body2)

/var/folders/84/skgmh_gx5bx_zpl3v_kjpwym0000gn/T/ipykernel_43750/2339947797.py:9: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res2=es.search(index="cpd",body=query_body2)


In [76]:
res2

ObjectApiResponse({'took': 8, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 158, 'relation': 'eq'}, 'max_score': 7.7753215, 'hits': [{'_index': 'cpd', '_id': '177', '_score': 7.7753215, '_source': {'section_title': 'Summary', 'text': 'Johns Hopkins Center for Health Security,Summary\nThe Johns Hopkins Center for Health Security (abbreviated CHS; previously the UPMC Center for Health Security, the Center for Biosecurity of UPMC, and the Johns Hopkins Center for Civilian Biodefense Strategies) is an independent, nonprofit organization of the Johns Hopkins Bloomberg School of Public Health, and part of the Environmental Health and Engineering department. It is concerned with the areas of health consequences from epidemics and disasters as well as averting biological weapons development, and implications of biosecurity for the bioeconomy. It is a think tank that does policy research and gives policy recommendations to t

In [93]:
query_body3={'query':
                 {'bool':
                      {'must':
                           {'match':{'text':'Spanish flu'}},
                      'must_not':
                           {'match':{'section_title':'External links'}},
                      'should':
                           {'match':{'section_title':{'query': 'Summary', 'boost': 1.5}}}
                      }
                 }
            }
res3=es.search(index="cpd",body=query_body3)

/var/folders/84/skgmh_gx5bx_zpl3v_kjpwym0000gn/T/ipykernel_43750/826147375.py:12: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res3=es.search(index="cpd",body=query_body3)


In [94]:
res3['hits']['total']['value']

43

In [95]:
res3

ObjectApiResponse({'took': 7, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 43, 'relation': 'eq'}, 'max_score': 12.732269, 'hits': [{'_index': 'cpd', '_id': '302', '_score': 12.732269, '_source': {'section_title': 'Summary', 'text': 'Swine influenza,Summary\nSwine influenza is an infection caused by any one of several types of swine influenza viruses. Swine influenza virus (SIV) or swine-origin influenza virus (S-OIV) is any strain of the influenza family of viruses that is endemic in pigs. As of 2009, the known SIV strains include influenza C and the subtypes of  influenza A known as H1N1, H1N2, H2N1, H3N1, H3N2, and H2N3.\nSwine influenza virus is common throughout pig populations worldwide. Transmission of the virus from pigs to humans is not common and does not always lead to human flu, often resulting only in the production of antibodies in the blood. If transmission does cause human flu, it is called zoonotic 

In [105]:
query_body4={'query':
                 {'bool':
                      {'must':
                           {'match':{'text':'Spanish flu'}},
                      'must_not':
                           {'match':{'section_title':'External links'}},
                      'should':
                           {'match':{'section_title':{'query': 'Summary', 'boost': 1.5}}}
                      }
                 },
             '_source':['text','section_title'],
             'highlight':
                 {'fields':
                     {'text':{},'section_title':{}}}
            }
res4=es.search(index="cpd",body=query_body4)

/var/folders/84/skgmh_gx5bx_zpl3v_kjpwym0000gn/T/ipykernel_43750/2112105697.py:16: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  res4=es.search(index="cpd",body=query_body4)


In [106]:
res4['hits']['total']['value']

43

In [107]:
res4

ObjectApiResponse({'took': 26, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 43, 'relation': 'eq'}, 'max_score': 12.732269, 'hits': [{'_index': 'cpd', '_id': '302', '_score': 12.732269, '_source': {'section_title': 'Summary', 'text': 'Swine influenza,Summary\nSwine influenza is an infection caused by any one of several types of swine influenza viruses. Swine influenza virus (SIV) or swine-origin influenza virus (S-OIV) is any strain of the influenza family of viruses that is endemic in pigs. As of 2009, the known SIV strains include influenza C and the subtypes of  influenza A known as H1N1, H1N2, H2N1, H3N1, H3N2, and H2N3.\nSwine influenza virus is common throughout pig populations worldwide. Transmission of the virus from pigs to humans is not common and does not always lead to human flu, often resulting only in the production of antibodies in the blood. If transmission does cause human flu, it is called zoonotic

In [110]:
#Unpacking the results here
docs=res4['hits']['hits']
docs

[{'_index': 'cpd',
  '_id': '302',
  '_score': 12.732269,
  '_source': {'section_title': 'Summary',
   'text': 'Swine influenza,Summary\nSwine influenza is an infection caused by any one of several types of swine influenza viruses. Swine influenza virus (SIV) or swine-origin influenza virus (S-OIV) is any strain of the influenza family of viruses that is endemic in pigs. As of 2009, the known SIV strains include influenza C and the subtypes of  influenza A known as H1N1, H1N2, H2N1, H3N1, H3N2, and H2N3.\nSwine influenza virus is common throughout pig populations worldwide. Transmission of the virus from pigs to humans is not common and does not always lead to human flu, often resulting only in the production of antibodies in the blood. If transmission does cause human flu, it is called zoonotic swine flu. People with regular exposure to pigs are at increased risk of swine flu infection.\nAround the mid-20th century, identification of influenza subtypes became possible, allowing accura